In [4]:
import pandas as pd
import numpy as np

In [5]:
pd.set_option('display.max_colwidth', -1)

## Get modeled item factors

In [6]:
comic_factors = pd.read_pickle('support_data/comics_factors.pkl')

## Get user's comics

In [18]:
comic_ids = [2171, 2637]

In [57]:
comic_factors.loc[2171]

features       [0.4914298355579376, 0.22184216976165771, -0.7872840762138367, -0.9905602335929871, -0.3622240126132965]
comic_title    Fables (Vertigo)                                                                                        
img_url        https://comrx.s3-us-west-2.amazonaws.com/covers/fables.jpg                                              
Name: 2171, dtype: object

In [58]:
comic_factors.loc[2637]

features       [0.08937276154756546, -0.12716685235500336, -0.8339032530784607, -0.9209315776824951, -0.5743098258972168]
comic_title    Gideon Falls (Image)                                                                                      
img_url        https://comrx.s3-us-west-2.amazonaws.com/covers/gideon_falls.jpg                                          
Name: 2637, dtype: object

In [16]:
comic_ids = [45, 13, 56]

In [59]:
comic_factors.loc[45]

features       [-0.6718752384185791, -0.5461212992668152, -0.8061680197715759, -0.3349294662475586, -0.588319718837738]
comic_title    4 Kids Walk Into a Bank (Other)                                                                         
img_url        https://comrx.s3-us-west-2.amazonaws.com/covers/4_kids_walk_into_a_bank.jpg                             
Name: 45, dtype: object

In [60]:
comic_factors.loc[13]

features       [-0.660622775554657, -0.2366572469472885, -0.8634084463119507, -0.005920375231653452, 0.540023148059845]
comic_title    1985 Black Hole Repo (Other)                                                                            
img_url        https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg                                          
Name: 13, dtype: object

In [61]:
comic_factors.loc[56]

features       [0.5547576546669006, -0.04429847374558449, -0.9259609580039978, 0.5181196331977844, -0.7316436171531677]
comic_title    68 Scars (Image)                                                                                        
img_url        https://comrx.s3-us-west-2.amazonaws.com/covers/68_scars.jpg                                            
Name: 56, dtype: object

In [22]:
len(comic_ids)

2

In [31]:
comic_ids = [45]

## Get modeled item factors

In [8]:
comic_factors = pd.read_pickle('support_data/comics_factors.pkl')

In [37]:
comic_ids = [7126, 6105, 45, 2182]

## Candidate Functions

In [25]:
def make_n_comic_recommendations(comics, comic_factors, top_n):
    # Create item matrix
    comic_matrix = create_user_item_matrix(comic_ids=comics,
                                           comic_factors=comic_factors
                                          )
    
    # Create user matrix
    user_matrix = create_user_impl_rate_matrix(comic_ids=comics)
    
    # Create utility matrix
    utility_matrix = create_user_util_matrix(comic_matrix, user_matrix)
    
    # Update comic_factors dataframe for this user -> predicted scores!
    cf = comic_factors.copy()
    u = utility_matrix
    cf['pred'] = cf['features'].apply(lambda x: np.dot(x, u))
    
    # Get recommendations
    top_n_df = cf.sort_values(by=['pred'], ascending=False).head(top_n).copy()
    top_n_df.reset_index(inplace=True)
    top_n_df = top_n_df.loc[:,['comic_id','comic_title','img_url']].copy()
    
    return top_n_df

### Get recommendations!


In [14]:
top_n = 8

In [38]:
results = make_n_comic_recommendations(comic_ids, comic_factors, top_n)

results

,comic_id,comic_title,img_url
0,1004,Buffy the Vampire Slayer Spik (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/buffy_the_vampire_slayer_spik.jpg
1,1551,Dark Tower Gunslinger So Fel (Marvel),https://comrx.s3-us-west-2.amazonaws.com/covers/dark_tower_gunslinger_so_fel.jpg
2,5940,Strange Aeons (Other),https://comrx.s3-us-west-2.amazonaws.com/covers/strange_aeons.jpg
3,5107,Robert E Howards Savage Swor (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/_no_cover_.jpg
4,5386,Shadows On the Grave (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/shadows_on_the_grave.jpg
5,7088,X-Files Conspiracy Transforme (IDW),https://comrx.s3-us-west-2.amazonaws.com/covers/x-files_conspiracy_transforme.jpg
6,3121,House of Mystery (Vertigo),https://comrx.s3-us-west-2.amazonaws.com/covers/house_of_mystery.jpg
7,4831,Predator Fire and Stone (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/predator_fire_and_stone.jpg


In [51]:
results = make_n_comic_recommendations([610,7088], comic_factors, 6)

results

,comic_id,comic_title,img_url
0,1004,Buffy the Vampire Slayer Spik (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/buffy_the_vampire_slayer_spik.jpg
1,6313,Titan (Other),https://comrx.s3-us-west-2.amazonaws.com/covers/titan.jpg
2,3121,House of Mystery (Vertigo),https://comrx.s3-us-west-2.amazonaws.com/covers/house_of_mystery.jpg
3,6726,Usagi Yojimbo (Dark Horse),https://comrx.s3-us-west-2.amazonaws.com/covers/usagi_yojimbo.jpg
4,5559,Snotgirl (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/snotgirl.jpg
5,4332,Motor Crush (Image),https://comrx.s3-us-west-2.amazonaws.com/covers/motor_crush.jpg


## Support Functions

In [24]:
def create_user_item_matrix(comic_ids, comic_factors):
    """
    Given list of user's comic preferences
    and a pandas df with item (comic) factors
    create a item matrix for the user
    """
    # Get rank
    num_latent_factors = len(comic_factors.features.iloc[0])
    
    # Initialize matrix
    comics_mtx = np.zeros(shape=(len(comic_ids), num_latent_factors)) 

    for index, comic in enumerate(comic_ids):
        #print(comic)
        #com_idx = comics_factors.loc[comics_factors['cxomic_id']==comic].index.values[0]
        comics_mtx[index, :] = np.array(comic_factors.loc[comic, 'features'])
    
    return comics_mtx

In [11]:
def create_user_impl_rate_matrix(comic_ids, ratings_list=None):
    """
    Given item matrix
    create implicit ratings matrix
    """
    if ratings_list is None:
        n = len(comic_ids)
        imp_rat_mtx = np.ones((n,1), 'int')
    else:
        imp_rat_mtx = np.array((ratings_list,)).T
    
    return imp_rat_mtx
    

In [12]:
def create_user_util_matrix(comics_matrix, user_rating_matrix):
    
    util_mtx = np.linalg.lstsq(comics_matrix, user_rating_matrix, rcond=None)

    # We just want the factors
    util_mtx = util_mtx[0].reshape((comics_matrix.shape[1],))

    return util_mtx